In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import os
import pandas as pd
import numpy as np
import pkg_resources
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
import scipy.stats as stats
from sklearn import metrics

%matplotlib inline

# autoreload makes it easier to interactively work on code in imported libraries
%load_ext autoreload
%autoreload 2

# Set pandas display options so we can read more of the comment text.
pd.set_option('max_colwidth', 300)

import model_bias_analysis

In [2]:
# Print python version
import sys
print(sys.version)

2.7.13 (default, Nov 24 2017, 17:33:09) 
[GCC 6.3.0 20170516]


# Load and pre-process Madlibs data

In [3]:
# Read madlibs dataset, convert Label to Booleans
madlibs_df = pd.read_csv('eval_datasets/bias_madlibs_77k_scored_prod_models.csv')
madlibs_df.loc[madlibs_df.Label == 'BAD', 'Label'] = True
madlibs_df.loc[madlibs_df.Label == 'NOT_BAD', 'Label'] = False
# Needed for AUC metrics to work properly
madlibs_df['Label'] = madlibs_df['Label'].astype(bool)

In [4]:
# Add boolean identity columns, e.g. "hug american" should have True in the "american" column.
madlibs_identity_terms = model_bias_analysis.read_identity_terms('bias_madlibs_data/adjectives_people.txt')
model_bias_analysis.add_subgroup_columns_from_text(madlibs_df, 'Text', madlibs_identity_terms)
madlibs_df.head()

,Label,Template,Text,id,RockV6_2:TOXICITY,Rock:TOXICITY,lesbian,gay,bisexual,transgender,...,older,young,younger,teenage,millenial,middle aged,elderly,blind,deaf,paralyzed
0,True,verb_adj,murder american,0,0.837852,0.811245,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,verb_adj,respect trans,1,0.066053,0.251806,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,verb_adj,respect homosexual,2,0.188938,0.831167,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,verb_adj,hug mexican,3,0.059981,0.098826,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,verb_adj,kill sikh,4,0.743793,0.888041,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Load and pre-process Civil Comments data

In [5]:
def convert_to_bool_column(df, src_column, dst_column):
    df.loc[df[src_column] >= 0.5, dst_column] = True
    df.loc[df[src_column] < 0.5, dst_column] = False
    # convert NaNs to False
    df.loc[df[src_column].isnull(), dst_column] = False
    # Needed for AUC metrics to work properly
    df[dst_column] = df[dst_column].astype(bool)

In [6]:
# Read Civil Comments dataset, set Label column to be a Boolean toxicity value.
civil_comments_df = pd.read_csv('eval_datasets/civil_comments_scored_prod_models.csv')
convert_to_bool_column(civil_comments_df, 'toxicity', 'Label')

In [7]:
# Convert identity terms to booleans
civil_comments_identity_terms = [
    'male', 'female', 'transgender', 'other_gender', 'heterosexual',
    'homosexual_gay_or_lesbian', 'bisexual', 'other_sexual_orientation', 'christian',
    'jewish', 'muslim', 'hindu', 'buddhist', 'atheist', 'other_religion', 'black',
    'white', 'asian', 'latino', 'other_race_or_ethnicity',
    'physical_disability', 'intellectual_or_learning_disability',
    'psychiatric_or_mental_illness', 'other_disability']
for identity in civil_comments_identity_terms:
    convert_to_bool_column(civil_comments_df, identity, identity)
civil_comments_df.head()
    

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,article_id,asian,atheist,bisexual,black,buddhist,christian,...,sexual_explicit,threat,toxicity,toxicity_annotator_count,transgender,white,wow,Rock:TOXICITY,RockV6_2:TOXICITY,Label
0,0,0,0,138380,False,False,False,False,False,False,...,0.000000,0.000000,0.783333,60,False,False,0,0.623230,0.865134,True
1,1,1,1,160937,False,False,False,False,False,False,...,0.000000,0.000000,0.628571,70,False,False,0,0.773702,0.831807,True
2,2,2,2,366992,False,False,False,False,False,False,...,0.028571,0.014286,0.571429,70,False,False,0,0.544393,0.607978,True
3,3,3,3,351749,False,False,False,False,False,False,...,0.000000,0.000000,0.675000,80,False,False,0,0.683339,0.869096,True
4,4,4,4,147521,False,False,False,False,False,False,...,0.000000,0.413333,0.800000,75,False,False,0,0.876304,0.836591,True


In [8]:
# Create another view of the data that is limited to the comments < 100 characters.
civil_comments_short_df = civil_comments_df.loc[civil_comments_df['Text'].str.len() < 100]

# Measure bias

In [ ]:
# Calculate bias metrics on each dataset
madlibs_bias_metrics = model_bias_analysis.compute_bias_metrics_for_models(
    madlibs_df,
    madlibs_identity_terms,
    ['Rock:TOXICITY', 'RockV6_2:TOXICITY'],
    'Label')
civil_comments_bias_metrics = model_bias_analysis.compute_bias_metrics_for_models(
    civil_comments_df,
    civil_comments_identity_terms,
    ['Rock:TOXICITY', 'RockV6_2:TOXICITY'],
    'Label')

In [ ]:
civil_comments_short_bias_metrics = model_bias_analysis.compute_bias_metrics_for_models(
    civil_comments_short_df,
    civil_comments_identity_terms,
    ['Rock:TOXICITY', 'RockV6_2:TOXICITY'],
    'Label')

In [ ]:
# Sort bias metrics by weighted pinned AUC
def sort_by_weighted_pinned_auc(bias_metrics):
    bias_metrics['weighted_pinned_auc'] = (
        bias_metrics['Rock:TOXICITY_bnsp_auc'] +
        bias_metrics['Rock:TOXICITY_bpsn_auc'] +
        bias_metrics['Rock:TOXICITY_subgroup_auc']) / 3
    bias_metrics = bias_metrics.sort_values('weighted_pinned_auc', ascending=True)
    return bias_metrics.drop('weighted_pinned_auc', axis=1)

madlibs_bias_metrics = sort_by_weighted_pinned_auc(madlibs_bias_metrics)
#civil_comments_bias_metrics = sort_by_weighted_pinned_auc(civil_comments_bias_metrics)
#civil_comments_short_bias_metrics = sort_by_weighted_pinned_auc(civil_comments_short_bias_metrics)

In [ ]:
civil_comments_short_bias_metrics

In [ ]:
# Define functions for printing bias metric heatmaps

N_AEG_COLORS = 12

aeg_negative_colors = sns.cubehelix_palette(N_AEG_COLORS,  # n_colors
                                   0.6,  # start at magenta
                                   0,  # no rotation
                                   3,  # gamma
                                   1,  # dark
                                   1,  # light
                                   reverse=True,
                                   as_cmap=False)[int(N_AEG_COLORS/2):]

aeg_positive_colors = sns.cubehelix_palette(N_AEG_COLORS,  # n_colors
                                   2,  # greenish
                                   0,  # no rotation
                                   3,  # gamma
                                   1,  # dark
                                   1,  # light
                                   reverse=False,
                                   as_cmap=False)[:int(N_AEG_COLORS/2)]
aeg_colors = aeg_negative_colors + aeg_positive_colors

def limit_subgroups(bias_metrics, subgroups):
    return bias_metrics.loc[bias_metrics.subgroup.isin(subgroups)]

def print_auc_heatmap(bias_metrics, subgroups, model, out=None, color_palette=None):
    bias_metrics_for_subgroups = limit_subgroups(bias_metrics, subgroups)
    if not color_palette:
        color_palette = sns.cubehelix_palette(100,  # n_colors
                                              0.6,  # start at magenta
                                              0,  # no rotation
                                              3,  # gamma
                                              1,  # dark
                                              1.2,  # light
                                              reverse=True,
                                              as_cmap=False)
    model_bias_analysis.plot_auc_heatmap(bias_metrics_for_subgroups, [model], color_palette, out=out)

def print_aeg_heatmap(bias_metrics, subgroups, model, out=None, color_palette=None):
    bias_metrics_for_subgroups = limit_subgroups(bias_metrics, subgroups)
    if not color_palette:
        color_palette = aeg_colors
    model_bias_analysis.plot_aeg_heatmap(bias_metrics_for_subgroups, [model], color_palette, out=out)

def get_identities_over_n(df, n, all_identities):
    results = []
    for identity in all_identities:
        num_records = len(df.query(identity + '==True'))
        if num_records > 100:
            results.append(identity)
    return results

In [ ]:
# Print Civil Comments heatmaps.  Filter identities to only use those which have >= 100
# short comments written about them.
CIVIL_COMMENTS_HEATMAP_IDENTITIES = get_identities_over_n(civil_comments_short_df, 100, civil_comments_identity_terms)
print_auc_heatmap(civil_comments_bias_metrics, CIVIL_COMMENTS_HEATMAP_IDENTITIES, 'Rock:TOXICITY', '/tmp/civil_aucs_1.png')
print_auc_heatmap(civil_comments_bias_metrics, CIVIL_COMMENTS_HEATMAP_IDENTITIES, 'RockV6_2:TOXICITY', '/tmp/civil_aucs_6.png')
print_aeg_heatmap(civil_comments_bias_metrics, CIVIL_COMMENTS_HEATMAP_IDENTITIES, 'Rock:TOXICITY', '/tmp/civil_aegs_1.png')
print_aeg_heatmap(civil_comments_bias_metrics, CIVIL_COMMENTS_HEATMAP_IDENTITIES, 'RockV6_2:TOXICITY', '/tmp/civil_aegs_6.png')

# Print Civil Comments Short Comments heatmaps
print_auc_heatmap(civil_comments_short_bias_metrics, CIVIL_COMMENTS_HEATMAP_IDENTITIES, 'Rock:TOXICITY', '/tmp/civil_short_aucs_1.png')
print_auc_heatmap(civil_comments_short_bias_metrics, CIVIL_COMMENTS_HEATMAP_IDENTITIES, 'RockV6_2:TOXICITY', '/tmp/civil_short_aucs_6.png')
print_aeg_heatmap(civil_comments_short_bias_metrics, CIVIL_COMMENTS_HEATMAP_IDENTITIES, 'Rock:TOXICITY', '/tmp/civil_short_aegs_1.png')
print_aeg_heatmap(civil_comments_short_bias_metrics, CIVIL_COMMENTS_HEATMAP_IDENTITIES, 'RockV6_2:TOXICITY', '/tmp/civil_short_aegs_6.png')

# Print Madlibs heatmaps, using same number of identities as civil comments
num_identities = 20
MADLIB_HEATMAP_IDENTITIES = madlibs_identity_terms[:num_identities]
print_auc_heatmap(madlibs_bias_metrics, MADLIB_HEATMAP_IDENTITIES, 'Rock:TOXICITY', '/tmp/madlibs_aucs_1.png')
print_auc_heatmap(madlibs_bias_metrics, MADLIB_HEATMAP_IDENTITIES, 'RockV6_2:TOXICITY', '/tmp/madlibs_aucs_6.png')
print_aeg_heatmap(madlibs_bias_metrics, MADLIB_HEATMAP_IDENTITIES, 'Rock:TOXICITY', '/tmp/madlibs_aegs_1.png')
print_aeg_heatmap(madlibs_bias_metrics, MADLIB_HEATMAP_IDENTITIES, 'RockV6_2:TOXICITY', '/tmp/madlibs_aegs_6.png')

In [ ]:
def get_percent_toxic(df):
    toxic_comments = df.query('Label == True')
    return len(toxic_comments) / len(df)
    

def print_count_and_percent_toxic(df, identity):
  # Query all training comments where the identity column equals True.
  identity_comments = df.query(identity + ' == True')

  # Query which of those comments also have "toxicity" equals True
  toxic_identity_comments = identity_comments.query('toxicity == True')
  # Alternatively you could also write a query using & (and), e.g.:
  # toxic_identity_comments = train_df.query(identity  + ' == True & toxicity == True')

  # Print the results.
  num_comments = len(identity_comments)
  percent_toxic = get_percent_toxic(identity_comments)
  print('%d comments refer to the %s identity, %.2f%% are toxic' % (
    num_comments,
    identity,
    # multiply percent_toxic by 100 for easier reading.
    100 * percent_toxic))

for identity in civil_comments_identity_terms:
    print_count_and_percent_toxic(civil_comments_df, identity)
    
# Print the results for all comments
num_comments = len(civil_comments_df)
percent_toxic = get_percent_toxic(civil_comments_df)
print('%d comments for all identities, %.2f%% are toxic' % (
    num_comments,
    # multiply percent_toxic by 100 for easier reading.
    100 * percent_toxic))